In [ ]:
%pip install openai

In [1]:
import pandas as pd
import json
import numpy as np
from time import sleep
import time
from openai import OpenAI
from tqdm import tqdm

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=open("API_key", "r").read(),
)

In [2]:
# Load the test datasets from the GitHub repositories (access to them is obtained by request to the AGILE repository owner)

en_ginco = pd.read_json("../../datasets/EN-GINCO-test-dataset/EN-GINCO.jsonl", lines=True)
x_ginco = pd.read_json("../../datasets/X-GINCO-test-set/X-GINCO.jsonl", lines=True)

print(en_ginco.shape, x_ginco.shape)

(272, 4) (790, 6)


In [3]:
def predict_gpt(df_test_name, gpt_model):

	dfs = {
		"en-ginco": en_ginco,
		"x-ginco": x_ginco
	}

	df = dfs[df_test_name]

	responses = []
	
	texts = df["text"].to_list()

	labels_dict = {
		0: "Other",
		1: "Information/Explanation",
		2: "News",
		3: "Instruction",
		4: "Opinion/Argumentation",
		5: "Forum",
		6: "Prose/Lyrical",
		7: "Legal",
		8: "Promotion"
	}

	label_dict_with_description_ext = {
			"Information/Explanation - An objective text that describes or presents an event, a person, a thing, a concept etc. Its main purpose is to inform the reader about something. Common features: objective/factual, explanation/definition of a concept (x is …), enumeration. E.g., research article, encyclopedia article, informational blog, product specification, course materials, general information, job description, manual, horoscope, travel guide, glossaries, historical article, biographical story/history.": 1,
			"News - An objective or subjective text which reports on an event recent at the time of writing or coming in the near future. Common features: adverbs/adverbial clauses of time and/or place (dates, places), many proper nouns, direct or reported speech, past tense. E.g., news report, sports report, travel blog, reportage, police report, announcement.": 2,
			"Instruction - An objective text which instructs the readers on how to do something. Common features: multiple steps/actions, chronological order, 1st person plural or 2nd person, modality (must, have to, need to, can, etc.), adverbial clauses of manner (in a way that), of condition (if), of time (after …). E.g., how-to texts, recipes, technical support.": 3,
			"Opinion/Argumentation - A subjective text in which the authors convey their opinion or narrate their experience. It includes promotion of an ideology and other non-commercial causes. This genre includes a subjective narration of a personal experience as well. Common features: adjectives/adverbs that convey opinion, words that convey (un)certainty (certainly, surely), 1st person, exclamation marks. E.g., review, blog (personal blog, travel blog), editorial, advice, letter to editor, persuasive article or essay, formal speech, pamphlet, political propaganda, columns, political manifesto.": 4,
			"Forum - A text in which people discuss a certain topic in form of comments. Common features: multiple authors, informal language, subjective (the writers express their opinions), written in 1st person. E.g., discussion forum, reader/viewer responses, QA forum.": 5,
			"Prose/Lyrical - A literary text that consists of paragraphs or verses. A literary text is deemed to have no other practical purpose than to give pleasure to the reader. Often the author pays attention to the aesthetic appearance of the text. It can be considered as art. E.g., lyrics, poem, prayer, joke, novel, short story. ": 6,
			"Legal - An objective formal text that contains legal terms and is clearly structured. The name of the text type is often included in the headline (contract, rules, amendment, general terms and conditions, etc.). Common features: objective/factual, legal terms, 3rd person. E.g., small print, software license, proclamation, terms and conditions, contracts, law, copyright notices, university regulation.": 7,
			"Promotion - A subjective text intended to sell or promote an event, product, or service. It addresses the readers, often trying to convince them to participate in something or buy something. Common features: contains adjectives/adverbs that promote something (high-quality, perfect, amazing), comparative and superlative forms of adjectives and adverbs (the best, the greatest, the cheapest), addressing the reader (usage of 2nd person), exclamation marks. E.g., advertisement, promotion of a product (e-shops), promotion of an accommodation, promotion of company's services, invitation to an event.": 8,
			"Other - A text that which does not fall under any of other genre categories.": 0,

	}

	start_time = time.time()

	for text in texts:
		completion = client.chat.completions.create(model=gpt_model,
		response_format= {"type": "json_object"},
		messages= [
		{
			"role": "user",
			"content": f"""
			### Task
			Your task is to classify the following text according to genre. Genres are text types, defined by the function of the text, author’s purpose and form of the text. Always provide a label, even if you are not sure.

			### Output format
				Return a valid JSON dictionary with the following key: 'genre' and a value should be an integer which represents one of the labels according to the following dictionary: {label_dict_with_description_ext}.

				
				Text: '{text}'
		"""
			}
		],
		temperature = 0)

		response=completion.choices[0].message.content

		response = response.replace("\n", "")
		response = response.replace("\t", "")

		# Get out a label
		try:
			# Convert the string into a dictionary
			response = json.loads(response)
			predicted = labels_dict[response["genre"]]
			responses.append(predicted)
		# add a possibility of something going wrong
		except:
			print("error with extracting a label")
			print(response)
			responses.append(response)

	end_time = time.time()
	elapsed_time_min = end_time-start_time

	print(f"Prediction finished. It took {elapsed_time_min/60} min for {df.shape[0]} instances - {elapsed_time_min/df.shape[0]} s per instance.")

	# Create a json with results

	current_results = {
		"system": gpt_model,
		"predictions": [
			{
			"train": "NA (zero-shot)",
			"test": "{}".format(df_test_name),
			"predictions": responses,
			}
		]
		}

	# The only thing that needs to be changed in the code from OpenAI
	gpt_model_name = gpt_model.split("/")[1]

	# Save the results as a new json
	with open("submissions/submission-{}-{}.json".format(gpt_model_name, df_test_name), "w") as file:
		json.dump(current_results, file)

	print("Classification with {} on {} finished.".format(gpt_model_name, df_test_name))

In [4]:
models = ["google/gemini-2.5-flash", "mistralai/mistral-medium-3.1"]

In [6]:
# Continue with prediction
for test in ["en-ginco"]:
	for model in ["mistralai/mistral-medium-3.1"]:
		print(model)
		predict_gpt(test, model)

mistralai/mistral-medium-3.1
Prediction finished. It took 3.6682628830273947 min for 272 instances - 0.8091756359619253 s per instance.
Classification with mistral-medium-3.1 on en-ginco finished.


In [7]:
for test in ["x-ginco"]:
	for model in models:
		print(model)
		predict_gpt(test, model)

google/gemini-2.5-flash
Prediction finished. It took 6.5814470489819845 min for 790 instances - 0.4998567378973659 s per instance.
Classification with gemini-2.5-flash on x-ginco finished.
mistralai/mistral-medium-3.1
Prediction finished. It took 7.830524067083995 min for 790 instances - 0.5947233468671388 s per instance.
Classification with mistral-medium-3.1 on x-ginco finished.


In [8]:
# Evaluate also Gemini 2.5 Pro
for test in ["en-ginco", "x-ginco"]:
	model = "google/gemini-2.5-pro"
	print(model)
	predict_gpt(test, model)

google/gemini-2.5-pro
Prediction finished. It took 46.625453011194864 min for 272 instances - 10.28502639952828 s per instance.
Classification with gemini-2.5-pro on en-ginco finished.
google/gemini-2.5-pro
error with extracting a label

error with extracting a label

error with extracting a label

error with extracting a label

error with extracting a label

Prediction finished. It took 148.74551327228545 min for 790 instances - 11.297127590300162 s per instance.
Classification with gemini-2.5-pro on x-ginco finished.
